# ecModel simulation tests

This notebook contains tests to see if simulating the ecModel with proteomics works as intended in the simulations service of DeCaF.

Note: using the requirements from [simulations](https://github.com/DD-DeCaF/simulations) (i.e. create a conda env and in it `pip install -r ./requirements/requirements.txt`)

Benjamín J. Sánchez, 2019-11-21

## 1. Loading Model and Data

In [1]:
import cobra
import os
import pandas as pd
import wget

#E. coli model:
wget.download("https://raw.githubusercontent.com/BenjaSanchez/notebooks/master/e_coli_simulations/eciML1515.xml")
ecModel = cobra.io.read_sbml_model("eciML1515.xml")
os.remove("eciML1515.xml")

#Yeast model:
#wget.download("https://raw.githubusercontent.com/SysBioChalmers/ecModels/master/ecYeastGEM/model/ecYeastGEM.xml")
#ecModel = cobra.io.read_sbml_model("ecYeastGEM.xml")
#os.remove("ecYeastGEM.xml")

 37% [...........................                                               ] 2957312 / 7926767

100% [..........................................................................] 7926767 / 7926767

In [2]:
#proteomics data:
def reset_proteomics():
    #data = pd.read_csv("ecoli_data_heinemann2016.csv")  # E. coli
    data = pd.read_csv("yeast_ref_data.csv")  # yeast
    proteomics = []
    for key, value in data.iterrows():
        protein = {"identifier":value[0], "measurement":value[1], "uncertainty":0}
        proteomics.append(protein)
    return(proteomics)

proteomics = reset_proteomics()
print(proteomics[:10])
print(len(proteomics))

[{'identifier': 'A5Z2X5', 'measurement': 6.34e-05, 'uncertainty': 0}, {'identifier': 'O13297', 'measurement': 1.91e-07, 'uncertainty': 0}, {'identifier': 'O13516', 'measurement': 5.33e-06, 'uncertainty': 0}, {'identifier': 'O43137', 'measurement': 2.4e-06, 'uncertainty': 0}, {'identifier': 'P00128', 'measurement': 4.07e-05, 'uncertainty': 0}, {'identifier': 'P00358', 'measurement': 6.3e-06, 'uncertainty': 0}, {'identifier': 'P00359', 'measurement': 0.00018932200000000001, 'uncertainty': 0}, {'identifier': 'P00410', 'measurement': 1.22e-05, 'uncertainty': 0}, {'identifier': 'P00546', 'measurement': 4.09e-06, 'uncertainty': 0}, {'identifier': 'P00549', 'measurement': 3.68e-05, 'uncertainty': 0}]
1221


In [3]:
#uptake data:
uptake = []
glucose = {"identifier":"r_1714_REV", "measurement":1, "uncertainty":0}
uptake.append(glucose)

## 2. Functions

In [4]:
from collections import namedtuple
from cobra.medium.boundary_types import find_external_compartment
from cobra.io.dict import reaction_to_dict

def apply_medium(model, is_ec_model, medium):
    operations = []
    warnings = []
    errors = []

    # Convert the list of dicts to a set of namedtuples to avoid duplicates, as
    # looking up metabolites in the model is a somewhat expensive operation.
    Compound = namedtuple("Compound", ["id", "namespace"])
    medium = set(Compound(id=c["identifier"], namespace=c["namespace"]) for c in medium)

    # Add trace metals
    medium.update(
        [
            Compound(id="CHEBI:25517", namespace="chebi"),
            Compound(id="CHEBI:25368", namespace="chebi"),
        ]
    )

    try:
        extracellular = find_external_compartment(model)
    except RuntimeError as error:
        # cobrapy throws RuntimeError if it for any reason is unable to find an
        # external compartment. See:
        # https://github.com/opencobra/cobrapy/blob/95d920d135fa824e6087f1fcbc88d50882da4dab/cobra/medium/boundary_types.py#L26
        message = (
            f"Cannot find an external compartment in model {model.id}: {str(error)}"
        )
        errors.append(message)
        # Cannot continue without knowing the external compartment, so
        # immediately return the error.
        return operations, warnings, errors

    # Create a map of exchange reactions and corresponding fluxes to apply to
    # the medium.
    medium_mapping = {}
    for compound in medium:
        print(compound)
        try:
            extracellular_metabolite = find_metabolite(
                model, compound.id, compound.namespace, extracellular
            )
        except MetaboliteNotFound:
            warning = (
                f"Cannot add medium compound '{compound.id}' - metabolite not found in "
                f"extracellular compartment '{extracellular}'"
            )
            warnings.append(warning)
        else:
            exchange_reactions = extracellular_metabolite.reactions.intersection(
                model.exchanges
            )
            if is_ec_model and len(exchange_reactions) == 2:
                exchange_reactions = get_ec_exchange_reaction(exchange_reactions, True)
            if len(exchange_reactions) != 1:
                errors.append(
                    f"Medium compound metabolite '{extracellular_metabolite.id}' has "
                    f"{len(exchange_reactions)} exchange reactions in the model; "
                    f"expected 1"
                )
                continue
            exchange_reaction = next(iter(exchange_reactions))

            # If someone already figured out the uptake rate for the compound, it's
            # likely more accurate than our assumptions, so keep it
            if exchange_reaction.id in model.medium:
                medium_mapping[exchange_reaction.id] = model.medium[
                    exchange_reaction.id
                ]
                continue

            if not extracellular_metabolite.formula:
                warning = (
                    f"No formula for metabolite '{extracellular_metabolite.id}', cannot"
                    f" check if it is a carbon source"
                )
                warnings.append(warning)
                # If we don't know, it's most likely that the metabolite does not have a
                # higher uptake rate than a carbon source, so set the bound still to 10
                medium_mapping[exchange_reaction.id] = 10
            elif "C" in extracellular_metabolite.elements:
                # Limit the uptake rate for carbon sources to 10
                medium_mapping[exchange_reaction.id] = 10
            else:
                medium_mapping[exchange_reaction.id] = 1000

    # Apply the medium to the model, letting cobrapy deal with figuring out the correct
    # bounds to change
    model.medium = medium_mapping

    # Add all exchange reactions to operations, to make sure any changed bounds is
    # properly updated
    for reaction in model.exchanges:
        operations.append(
            {
                "operation": "modify",
                "type": "reaction",
                "id": reaction.id,
                "data": reaction_to_dict(reaction),
            }
        )

    return operations, warnings, errors


def apply_measurements(
    model,
    biomass_reaction,
    is_ec_model,
    fluxomics,
    metabolomics,
    proteomics,
    uptake_secretion_rates,
    molar_yields,
    growth_rate,
):
    operations = []
    warnings = []
    errors = []

    def bounds(measurement, uncertainty):
        """Return resolved bounds based on measurement and uncertainty"""
        if uncertainty:
            return (measurement - uncertainty, measurement + uncertainty)
        else:
            return (measurement, measurement)

    # If an enzyme constrained model with proteomics was supplied, flexibilize the
    # proteomics data and redefine the growth rate based on simulations.
    if growth_rate and proteomics and is_ec_model:
        growth_rate, proteomics, prot_warnings = flexibilize_proteomics(
            model, biomass_reaction, growth_rate, proteomics
        )
        for warning in prot_warnings:
            warnings.append(warning)

    # Constrain the model with the observed growth rate
    if growth_rate:
        reaction = model.reactions.get_by_id(biomass_reaction)
        reaction.bounds = bounds(growth_rate["measurement"], growth_rate["uncertainty"])
        operations.append(
            {
                "operation": "modify",
                "type": "reaction",
                "id": reaction.id,
                "data": reaction_to_dict(reaction),
            }
        )

    for measure in fluxomics:
        try:
            reaction = model.reactions.get_by_id(measure["identifier"])
        except KeyError:
            errors.append(
                f"Cannot find reaction '{measure['identifier']}' in the model"
            )
        else:
            reaction.bounds = bounds(measure["measurement"], measure["uncertainty"])
            operations.append(
                {
                    "operation": "modify",
                    "type": "reaction",
                    "id": reaction.id,
                    "data": reaction_to_dict(reaction),
                }
            )

    for metabolite in metabolomics:
        warning = (
            f"Cannot apply metabolomics measure for '{metabolite['identifier']}'; "
            f"feature has not yet been implemented"
        )
        warnings.append(warning)

    for measure in proteomics:
        if is_ec_model:
            try:
                reaction = model.reactions.get_by_id(
                    f"prot_{measure['identifier']}_exchange"
                )
            except KeyError:
                warning = f"Cannot find protein '{measure['identifier']}' in the model"
                warnings.append(warning)
            else:
                # measurement only modifies the upper bound (enzymes can be unsaturated)
                lb, ub = bounds(measure["measurement"], measure["uncertainty"])
                reaction.bounds = 0, ub
                operations.append(
                    {
                        "operation": "modify",
                        "type": "reaction",
                        "id": reaction.id,
                        "data": reaction_to_dict(reaction),
                    }
                )
        else:
            warning = (
                f"Cannot apply proteomics measurements for "
                f"non enzyme-constrained model {model.id}"
            )
            warnings.append(warning)
            break

    for rate in uptake_secretion_rates:
        try:
            metabolite = find_metabolite(
                model, rate["identifier"], rate["namespace"], "e"
            )
        except MetaboliteNotFound as error:
            errors.append(str(error))
        else:
            exchange_reactions = metabolite.reactions.intersection(model.exchanges)
            if is_ec_model and len(exchange_reactions) == 2:
                exchange_reactions = get_ec_exchange_reaction(
                    exchange_reactions, rate["measurement"] < 0
                )
            if len(exchange_reactions) != 1:
                errors.append(
                    f"Measured metabolite '{metabolite['identifier']}' has "
                    f"{len(exchange_reactions)} exchange reactions in the model; "
                    f"expected 1"
                )
                continue
            exchange_reaction = next(iter(exchange_reactions))
            lower_bound, upper_bound = bounds(rate["measurement"], rate["uncertainty"])

            # data is adjusted assuming a forward exchange reaction, i.e. x -->
            # (sign = -1), so if we instead actually have --> x, then multiply with -1
            direction = exchange_reaction.metabolites[metabolite]
            if direction > 0:
                lower_bound, upper_bound = -1 * lower_bound, -1 * upper_bound
            exchange_reaction.bounds = lower_bound, upper_bound
            operations.append(
                {
                    "operation": "modify",
                    "type": "reaction",
                    "id": exchange_reaction.id,
                    "data": reaction_to_dict(exchange_reaction),
                }
            )

    for molar_yield in molar_yields:
        warning = (
            f"Cannot apply molar yield measurement for '"
            f"{molar_yield['product_identifier']}/{molar_yield['substrate_identifier']}"
            f"'; feature has not yet been implemented"
        )
        warnings.append(warning)
    return operations, warnings, errors


def flexibilize_proteomics(model, biomass_reaction, growth_rate, proteomics):
    # reset growth rate in model:
    model.reactions.get_by_id(biomass_reaction).bounds = (0, 1000)

    # build a table with protein ids, met ids in model and values to constrain with:
    prot_df = pd.DataFrame()
    for protein in proteomics:
        protein_id = protein["identifier"]
        lb, ub = bounds(protein["measurement"], protein["uncertainty"])
        for met in model.metabolites.query(lambda m: protein_id in m.id):
            new_row = pd.DataFrame(
                data={"met_id": met.id, "value": ub}, index=[protein_id]
            )
            prot_df = prot_df.append(new_row)

    # constrain the model with all proteins and optimize:
    limit_proteins(model, prot_df["value"])
    solution = model.optimize()
    new_growth_rate = solution.objective_value

    # while the model cannot grow to the desired level, remove the protein with
    # the highest shadow price:
    minimal_growth, ub = bounds(growth_rate["measurement"], growth_rate["uncertainty"])
    prots_to_remove = []
    while new_growth_rate < minimal_growth and not prot_df.empty:
        # get most influential protein in model:
        top_protein = top_shadow_prices(solution, list(prot_df["met_id"]))
        value = top_protein[top_protein.index[0]]
        top_protein = top_protein.index[0]
        top_protein = prot_df.index[prot_df["met_id"] == top_protein][0]
        print("working: " + top_protein + " (sp=" + str(value) + ") - mu = " + str(new_growth_rate))
        
        # update data: append protein to list, remove from current dataframe and
        # increase the corresponding upper bound to +1000:
        prots_to_remove.append(top_protein)
        prot_df = prot_df.drop(labels=top_protein)
        limit_proteins(model, pd.Series(data=[1000], index=[top_protein]))

        # re-compute solution:
        solution = model.optimize()
        if solution.objective_value == new_growth_rate:  # the algorithm is stuck
            break
        new_growth_rate = solution.objective_value

    # update growth rate if optimization was not successful:
    if new_growth_rate < minimal_growth:
        if growth_rate["uncertainty"]:
            growth_rate["measurement"] = new_growth_rate + growth_rate["uncertainty"]
        else:
            growth_rate["measurement"] = new_growth_rate

    # update proteomics by removing flexibilized proteins:
    for protein in prots_to_remove:
        index = next(
            (
                index
                for (index, dic) in enumerate(proteomics)
                if dic["identifier"] == protein
            ),
            None,
        )
        del proteomics[index]

    return growth_rate, proteomics


def limit_proteins(model, measurements):
    for protein_id, measure in measurements.items():
        try:
            rxn = model.reactions.get_by_id(f"prot_{protein_id}_exchange")
        except KeyError:
            pass
        else:
            # update only upper_bound (as enzymes can be unsaturated):
            rxn.bounds = (0, measure)
    return


def top_shadow_prices(solution, met_ids, top=1):
    shadow_pr = solution.shadow_prices
    shadow_pr = shadow_pr.loc[shadow_pr.index.isin(met_ids)]
    return shadow_pr.sort_values()[:top]


def bounds(measurement, uncertainty):
    if uncertainty:
        return measurement - uncertainty, measurement + uncertainty
    else:
        return measurement, measurement


def find_metabolite(model, id, namespace, compartment):
    def query_fun(metabolite):
        if metabolite.compartment != compartment:
            return False

        result = _query_item(metabolite, id, namespace)
        if result:
            return result

        # If the original query fails, retry with the compartment id appended
        # to the identifier (a regular convenation with BiGG metabolites, but
        # may also be the case in other namespaces).
        return _query_item(metabolite, f"{id}_{compartment}", namespace)

    metabolites = model.metabolites.query(query_fun)
    if len(metabolites) == 0:
        raise MetaboliteNotFound(
            f"Could not find metabolite {id} or {id}_{compartment} in "
            f"namespace {namespace} and compartment {compartment} for model "
            f"{model.id}"
        )
    elif len(metabolites) > 1:
        raise IndexError(f"Expected single metabolite, found {metabolites}")
    else:
        return metabolites[0]

def _query_item(item, query_id, query_namespace):
    # Try the default identifiers (without confirming the namespace)
    if query_id.lower() == item.id.lower():
        return True

    # Otherwise, try to find a case insensitive match for the namespace key
    for namespace in item.annotation:
        if query_namespace.lower() == namespace.lower():
            annotation = item.annotation[namespace]
            # Compare the identifier case insensitively as well
            # Annotations may contain a single id or a list of ids
            if isinstance(annotation, list):
                if query_id.lower() in [i.lower() for i in annotation]:
                    return True
            else:
                if query_id.lower() == annotation.lower():
                    return True
    return False

def compute_measurements(proteomics, ecModel):
    measurements = pd.DataFrame()
    for protein in proteomics:
        protein_id = protein["identifier"]
        lb, ub = bounds(protein["measurement"], protein["uncertainty"])
        for met in ecModel.metabolites:
            if protein_id in met.id:
                new_row = pd.DataFrame(data={"met_id": met.id, "value": ub}, index=[protein_id])
                measurements = measurements.append(new_row)
    return measurements

def get_ec_exchange_reaction(exchange_reactions, consumption):
    ec_exchange_reaction = []
    for reaction in exchange_reactions:
        if (reaction.products and consumption) or (reaction.reactants and not consumption):
            ec_exchange_reaction.append(reaction)
    return ec_exchange_reaction

class MetaboliteNotFound(Exception):
    pass

## 3. Testing

In [5]:
solution = ecModel.optimize()
print(solution)

<Solution 0.088 at 0x1b8e0e36388>


In [6]:
measurements = pd.DataFrame()
for protein in proteomics:
    protein_id = protein["identifier"]
    lb, ub = bounds(protein["measurement"], protein["uncertainty"])
    for met in ecModel.metabolites:
        if protein_id in met.id:
            new_row = pd.DataFrame(data={"met_id": met.id, "value": ub}, index=[protein_id])
            measurements = measurements.append(new_row)
print(measurements.head())

                          met_id     value
P00128  prot_P00128__91__c__93__  0.000041
P00358  prot_P00358__91__c__93__  0.000006
P00359  prot_P00359__91__c__93__  0.000189
P00410  prot_P00410__91__c__93__  0.000012
P00549  prot_P00549__91__c__93__  0.000037


In [7]:
limit_proteins(ecModel, measurements["value"])
solution = ecModel.optimize()
print(solution.objective_value)

5.199145593769825e-05


In [8]:
top_protein = top_shadow_prices(solution, list(measurements["met_id"]))
print(top_protein)

prot_P38604__91__c__93__   -189.05984
Name: shadow_prices, dtype: float64


In [9]:
#new_growth_rate, new_proteomics = flexibilize_proteomics(ecModel, "BIOMASS_Ec_iML1515_core_75p37M", {"measurement":0.8, "uncertainty":0.1}, proteomics)
growth_rate, proteomics = flexibilize_proteomics(ecModel, "r_2111", {"measurement":0.08, "uncertainty":0.01}, proteomics)

working: P38604 (sp=-189.0598397734483) - mu = 5.199145593769825e-05
working: P38972 (sp=-1761.0143524943996) - mu = 0.018666752136436607
working: P00931 (sp=-2074.8931245794574) - mu = 0.022201356433002306
working: Q12233 (sp=-4642.300091434081) - mu = 0.023065544305181932
working: P24521 (sp=-55317.86427789225) - mu = 0.01864212026164969
working: P07283 (sp=-4177.57190480627) - mu = 0.034757398247988174
working: Q06405 (sp=-4642.298754485921) - mu = 0.03710384909604921
working: Q12349 (sp=-4642.298694622616) - mu = 0.03854296121291276
working: P21306 (sp=-4642.298694622572) - mu = 0.043603066790051444
working: P25340 (sp=-67422.99117010784) - mu = 0.04146513956961635
working: P28777 (sp=-10112.224326534935) - mu = 0.05015663265961329
working: P10659 (sp=-15090.604451151283) - mu = 0.05839281321776375
working: P54839 (sp=-6752.045587652795) - mu = 0.0695460695528238


In [10]:
print(growth_rate)
print(len(proteomics))
solution = ecModel.optimize()
print(solution)

{'measurement': 0.08, 'uncertainty': 0.01}
1208
<Solution 0.073 at 0x1b8e10f12c8>


## 4. Signs of Shadow Prices

Let's evaluate what happens if we take the positive shadow prices (if any) instead of the negative ones, by sorting in a decreasing order instead of an increasing order:

In [11]:
def top_shadow_prices(solution, met_ids, top=1):
    shadow_pr = solution.shadow_prices
    shadow_pr = shadow_pr.loc[shadow_pr.index.isin(met_ids)]
    return shadow_pr.sort_values(ascending=False)[:top]

proteomics = reset_proteomics()
top_protein = top_shadow_prices(solution, list(measurements["met_id"]))
print(top_protein)
growth_rate, proteomics = flexibilize_proteomics(ecModel, "r_2111", {"measurement":0.08, "uncertainty":0.01}, proteomics)
print(growth_rate)
print(len(proteomics))
solution = ecModel.optimize()
print(solution)

prot_P36136__91__c__93__    152.853593
Name: shadow_prices, dtype: float64
working: P39979 (sp=0.0) - mu = 5.199145593769825e-05
{'measurement': 0.010051991455937699, 'uncertainty': 0.01}
1220
<Solution 0.000 at 0x1b8e0c697c8>


No positive shadow prices are detected, and hence the algorithm gets stuck. Finally, let's take the absolute value of all shadow prices and then rank in decreasing order, so that the sign doesn't matter:

In [12]:
def top_shadow_prices(solution, met_ids, top=1):
    shadow_pr = solution.shadow_prices
    shadow_pr = shadow_pr.loc[shadow_pr.index.isin(met_ids)]
    for met_id, sp in shadow_pr.items():
        shadow_pr[met_id] = abs(sp)
    return shadow_pr.sort_values(ascending=False)[:top]

proteomics = reset_proteomics()
top_protein = top_shadow_prices(solution, list(measurements["met_id"]))
print(top_protein)
growth_rate, proteomics = flexibilize_proteomics(ecModel, "r_2111", {"measurement":0.08, "uncertainty":0.01}, proteomics)
print(growth_rate)
print(len(proteomics))
solution = ecModel.optimize()
print(solution)

prot_P38604__91__c__93__    189.05984
Name: shadow_prices, dtype: float64
working: P38604 (sp=189.05983977344818) - mu = 5.199145593769825e-05
working: P38972 (sp=1760.993678094865) - mu = 0.01866653298779876
working: P24521 (sp=55317.864277892244) - mu = 0.01864212026164968
working: P00931 (sp=2074.8931245790336) - mu = 0.02220135643299561
working: Q12233 (sp=4642.298694622596) - mu = 0.02306553736504101
working: P07283 (sp=4177.57190480627) - mu = 0.03475739824798817
working: Q06405 (sp=4642.298814349269) - mu = 0.03710384957450237
working: Q12349 (sp=4642.298814349255) - mu = 0.03854296220695032
working: P21306 (sp=4642.298814349284) - mu = 0.04360306791459133
working: P25340 (sp=67422.99117010784) - mu = 0.04146513956961631
working: P28777 (sp=10112.22432653517) - mu = 0.05015663265961439
working: P10659 (sp=15090.604451151285) - mu = 0.0583928132177621
working: P54839 (sp=6752.045587652798) - mu = 0.06954606955282379
{'measurement': 0.08, 'uncertainty': 0.01}
1208
<Solution 0.073 

We see that we get the same results as with negative shadow prices, so no positive shadow prices are popping up in any of the flexibilization iterations.

## 5. Filtering _E. coli_ data

Temporal, while the data upload interface at caffeine is still wip.

In [ ]:
data = pd.read_csv("ecoli_data_heinemann2016.csv")
data_filtered = pd.DataFrame()
for key, value in data.iterrows():
    for met in ecModel.metabolites:
        if value[0] in met.id:
            data_filtered = data_filtered.append(data.loc[key])

data_filtered.to_csv(path_or_buf="ecoli_data_filtered.csv")